In [1]:
!git clone https://github.com/jack-eddy-symposium/risk-resiliency-spwx.git
# !du -sh /content/risk-resiliency-spwx/data

import numpy as np
import pandas as pd
import glob
import os

import datetime
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
!pip install sunpy
!pip install zeep
!pip install climextremes
!pip install drms

from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.timeseries import TimeSeries


from scipy.integrate import trapz



Cloning into 'risk-resiliency-spwx'...
remote: Enumerating objects: 3226, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 3226 (delta 54), reused 75 (delta 45), pack-reused 3135
Receiving objects: 100% (3226/3226), 312.64 MiB | 9.00 MiB/s, done.
Resolving deltas: 100% (507/507), done.
Updating files: 100% (2896/2896), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.3 MB/s eta 0:00:00


In [13]:
import os
import pandas as pd

# Base data directory containing event folders
base_data_path = '/content/risk-resiliency-spwx/data'

# List to collect max rows from all files in all events
max_row_list = []

# Loop over every event folder in the data directory
for event_folder in os.listdir(base_data_path):
    event_path = os.path.join(base_data_path, event_folder)

    # Check if it's a directory
    if os.path.isdir(event_path):
        # Construct path to the GIC subfolder within the event folder
        gic_path = os.path.join(event_path, 'GIC')

        # Check if the GIC subfolder exists
        if os.path.exists(gic_path) and os.path.isdir(gic_path):

            csv_files = [os.path.join(gic_path, f) for f in os.listdir(gic_path) if f.endswith('.csv')]
            csv_files.sort()  # Sort files by name

            # Exclude the last two CSV files
            csv_files_to_process = csv_files[:-2]

            # Process each CSV file
            for single_path in csv_files_to_process:
                df = pd.read_csv(single_path)
                idx = df['GICMeasured'].idxmax()
                max_row = df.loc[idx]
                max_row_list.append(max_row)

# Convert the list to a DataFrame
max_rows_df = pd.DataFrame(max_row_list)
max_rows_df


# Convert 'sampledatetime' column to datetime format
max_rows_df['SampleDateTime'] = pd.to_datetime(max_rows_df['SampleDateTime'])

# Extract the year from the 'sampledatetime' column
max_rows_df['year'] = max_rows_df['SampleDateTime'].dt.year

# Save the modified DataFrame to a new CSV file
max_rows_df.to_csv('max_rows_data.csv', index=False)

# Load the CSV file into a DataFrame
max_rows_df = pd.read_csv('max_rows_data.csv')

# Print
print(max_rows_df)

      GICDeviceID       SampleDateTime  GICMeasured  year
0           10063  2017-09-28 03:17:32         0.50  2017
1           10066  2017-09-27 18:08:10         1.35  2017
2           10067  2017-09-28 20:21:00        -1.13  2017
3           10071  2017-09-28 14:04:00         4.29  2017
4           10074  2017-09-28 07:00:30         2.08  2017
...           ...                  ...          ...   ...
1849        10356  2021-05-12 15:00:00         0.42  2021
1850        10357  2021-05-12 04:57:30        48.70  2021
1851        10359  2021-05-13 04:43:38         1.27  2021
1852        10441  2021-05-13 11:15:00        -5.73  2021
1853        10442  2021-05-12 02:40:05         3.67  2021

[1854 rows x 4 columns]


In [3]:
# def clean_GIC_data(df,df_sensor_locations,threshold):
#     '''
#     - delete repeated time points
#     - delete station if all time points are empty
#     - Find how many of the measurements differ from one time step to the next and delete station if on average measurements differ less than once in 5 minutes
#     - remove repeat stations from files
#     - remove stations with an unacceptable level of bias

#     '''

#     # Delete repeated time points
#     df = df.drop(df[df.duplicated(subset='datetimes', keep='first')].index)

#     cols_to_drop = []
#     for id_loop in df.columns.to_list():

#         if (id_loop == 'datetimes'):
#             continue

#         # Delete station if most time points are empty
#         if ( (df[id_loop].isna().mean() * 100) > 80.):
#             cols_to_drop.append(id_loop)
#             continue

#     print('cols_to_drop after first filter = {}'.format(cols_to_drop))

#     # remove repeat stations from files
#     duplicate_lats = df_sensor_locations.duplicated(subset=' Latitude',keep='first')
#     duplicate_lons = df_sensor_locations.duplicated(subset=' Longitude',keep='first')
#     duplicate_locations = df_sensor_locations['Device ID'].values[np.where( (duplicate_lats.values==True) & (duplicate_lons.values==True) )]

#     for d in duplicate_locations:
#         try:
#             #print(df[str(d)])
#             cols_to_drop.append(str(d))
#         except:
#             print('no df entry for devide ID {}'.format(str(d)))
# #         # Check I wrote to guarantee the code above removes duplicate stations (keeping only one of the same lat-long)
# #         for a in df_sensor_locations['Device ID'].values[np.where( (duplicate_lats.values==True) & (duplicate_lons.values==True) )]:
# #             idx = df_sensor_locations.index[df_sensor_locations['Device ID'] == a].tolist()
# #             print('the index of the duplicated lat and lon = {}'.format(a))
# #             print(' \t --> {}'.format( df_sensor_locations[idx[0]-2:idx[0]+3] ))

#     print('cols_to_drop after second filter = {}'.format(cols_to_drop))

#     # Find how many of the measurements differ from one time step to the next and delete station if measurements repeat for more than 60% of the data
#     for device_id in df.columns.to_list():
#         if device_id == 'datetimes':
#             continue
#         count_zeros = np.count_nonzero(np.diff(df[device_id].values) == 0)
#         tmp_len = len(np.argwhere(np.isfinite(df[device_id].values)==True))
#         tmp = (count_zeros/tmp_len)*100
# #             print('percentage of repeated values = {}'.format( tmp  ))
#         if tmp > 60.:
#             cols_to_drop.append(device_id)

#     print('cols_to_drop after third filter = {}'.format(cols_to_drop))



#     # Catch bias that is near the threshold level (would cause issues with burst statistics)
#     for device_id in df.columns.to_list():
#         if device_id == 'datetimes':
#             continue
#         med_tmp = np.nanmedian(df[device_id].values)
#         # apply conservative value for an unacceptable level of bias
#         if abs(med_tmp) >= (threshold/2):
#             cols_to_drop.append(device_id)

#     print('cols_to_drop after fourth filter = {}'.format(cols_to_drop))



#     cols_to_drop = list(set(cols_to_drop))
#     print('Total number of columns to drop = {}'.format(len(cols_to_drop)))
# #     # some stations were removed in the process of creating the integrated data files, so need to check if all columns to remove exist
#     cols_to_drop_final = []
#     for c in cols_to_drop:
#         if any( str(c) in x for x in df.columns.to_list()):
#             cols_to_drop_final.append(str(c))
#         else:
#             print('{} is NOT in the columns list'.format(c))

#     df = df.drop(cols_to_drop_final,axis=1)

#     return df

# def visualize_dataframe_columns(df):
#     fig, axs = plt.subplots(len(df.columns), 1, figsize=(6, 1.5*len(df.columns)))

#     for i, col in enumerate(df.columns):
#         axs[i].plot(df[col])
#         axs[i].set_ylabel(col, rotation=0, labelpad=20, fontsize=10)
#         axs[i].set_yticklabels([])
#         axs[i].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)

#     plt.tight_layout()
#     plt.show()

# def visualize_dataframe_singlecolumn(df,col_name):
#     fig, axs = plt.subplots(1, 1, figsize=(10,8))

#     axs.plot(df['datetimes'],df[col_name])
#     axs.set_ylabel(col_name, rotation=0, labelpad=20, fontsize=10)
# #     axs[i].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)

#     plt.tight_layout()
#     plt.show()

POT

In [38]:
import numpy as np
import climextremes
import pandas as pd

max_rows_df = pd.read_csv('max_rows_data.csv')

# Define the threshold for extreme precipitation events
threshold = 2

# Filter out values that exceed the threshold
exceedance_values = max_rows_df[max_rows_df['GICMeasured'] > threshold]

# Determine the range of years
first_year = min(max_rows_df['year'])
last_year = max(max_rows_df['year'])

# Define the years for predicting return values (next 100 years)
years_to_predict = np.arange(last_year + 1, last_year + 101)

# Perform extreme value analysis
stationary_fit = True

if stationary_fit:
    result = climextremes.fit_pot(np.array(exceedance_values['GICMeasured']),
                                  nBlocks=len(exceedance_values['year']),
                                  threshold=threshold,
                                  firstBlock=first_year,
                                  blockIndex=np.array(exceedance_values['year']),
                                  getParams=True,
                                  returnPeriod=100,
                                  returnValue=0.61,
                                  bootSE=False)

else:
    # Perform extreme value analysis with a non-stationary fit
    # may need to provide additional parameters for a non-stationary fit based on the requirements of the climextremes package
    pass

# Access the return values for the next 100 years
return_values_100_years = result['returnValue']
print("return value is: ", return_values_100_years)
print(result['se_returnValue'])       # return value standard error (asymptotic)
#print(result['se_returnValue_boot'])  # return value standard error (bootstrapping)


# The example code for reference
# firstYr = min(Fort.year)
# yrs = np.array(range(int(firstYr), int(max(Fort.year)+1)))
# nYrs = len(yrs)
# yrsToPred = np.array([min(Fort.year), max(Fort.year)])

# threshold = 0.395

# FortExc = Fort[Fort.Prec > threshold]

# # stationary fit
# result = climextremes.fit_pot(np.array(FortExc.Prec), nBlocks = nYrs, threshold = threshold, firstBlock = firstYr,
#blockIndex = np.array(FortExc.year), getParams = True, returnPeriod = 20, returnValue = 3.5, bootSE = True)
# result['returnValue']
# result['se_returnValue']       # return value standard error (asymptotic)
# result['se_returnValue_boot']  # return value standard error (bootstrapping)
# result['logReturnProb']        # log of probability of exceeding 'returnValue'
# result['mle']                  # MLE array
# result['mle_names']            # names for MLE array
# result['mle'][2]               # MLE for shape parameter

# result['numBootFailures']      # number of bootstrap datasets for which the model could not be fit; if this is non-negligible relative to the number of bootstrap samples (default of 250), interpret the bootstrap results with caution

# # nonstationary fit with location linear in year and two return values requested
# result = climextremes.fit_pot(np.array(FortExc.Prec), x = yrs, firstBlock = firstYr, nBlocks = nYrs, threshold = threshold,
#blockIndex = np.array(FortExc.year), locationFun = 1, getParams = True,
#returnPeriod = np.array([20, 30]), returnValue = 3.5, xNew = yrsToPred, bootSE = False)
# result['returnValue']
# result['se_returnValue']

# # fit with location a function of two covariates
# # here I'll use year and a random vector just to illustrate syntax
# # make 'x' be a 2-column numpy array, each column a covariate
# # 'xNew' also needs to have 2 columns, each row is a different set of covariate values
# tmp = np.random.rand(nYrs)
# covByBlock = np.c_[yrs, np.random.rand(nYrs)]
# result = climextremes.fit_pot(np.array(FortExc.Prec), x = covByBlock, firstBlock = firstYr, nBlocks = nYrs, threshold = threshold, blockIndex = np.array(FortExc.year), locationFun = np.array([1,2]), getParams = True, returnPeriod = 20, returnValue = 3.5, xNew = np.array([[min(Fort.year), 0], [max(Fort.year), 0]]), bootSE = False)

# # with declustering (using max of exceedances on contiguous days)
# result = climextremes.fit_pot(np.array(FortExc.Prec), x = yrs, firstBlock = firstYr, nBlocks = nYrs, threshold = threshold, blockIndex = np.array(FortExc.year), index = np.array(FortExc.obs), locationFun = 1, declustering = "noruns", getParams = True, returnPeriod = 20, returnValue = 3.5, xNew = yrsToPred, bootSE = False)
# result['returnValue']
# result['se_returnValue']

# # with declustering (consider sequential blocks of 5 days and only use the max of any exceedances within a block)
# result = climextremes.fit_pot(np.array(FortExc.Prec), x = yrs, firstBlock = firstYr, nBlocks = nYrs, threshold = threshold, blockIndex = np.array(FortExc.year), index = np.array(FortExc.obs), locationFun = 1, declustering = 5, getParams = True, returnPeriod = 20, returnValue = 3.5, xNew = yrsToPred, bootSE = False)
# result['returnValue']
# result['se_returnValue']

# # with replicates; for illustration here, I'll just duplicate the Fort data
# result = climextremes.fit_pot(np.append(np.array(FortExc.Prec), np.array(FortExc.Prec)), x = yrs, firstBlock = firstYr, nBlocks = nYrs, nReplicates = 2, threshold = threshold, blockIndex = np.append(FortExc.year, FortExc.year), locationFun = 1, getParams = True, returnPeriod = 20, returnValue = 3.5, xNew = yrsToPred, bootSE = False)
# result['returnValue']
# result['se_returnValue']

# # analysis of seasonal total precipitation
# tmp = Fort[np.logical_and(Fort['month'] < 9, Fort['month'] > 5)]
# FortSummerTotal = tmp.groupby('year').sum()[['Prec']]
# FortSummerTotal.reset_index(inplace=True)
# threshold = np.percentile(FortSummerTotal.Prec, 80)
# FortSummerTotalExc = FortSummerTotal[FortSummerTotal.Prec > threshold]

# result = climextremes.fit_pot(np.array(FortSummerTotalExc.Prec), x = yrs, firstBlock = firstYr, nBlocks = nYrs, blockIndex = np.array(FortSummerTotalExc.year), locationFun = 1, threshold = threshold, getParams = True, returnPeriod = 20, returnValue = 10, xNew = yrsToPred, bootSE = False)
# result['returnValue']
# result['se_returnValue']

# # modifying control arguments and seeing more information on the optimization
# result = climextremes.fit_pot(np.array(FortSummerTotalExc.Prec), x = yrs, firstBlock = firstYr, nBlocks = nYrs, blockIndex = np.array(FortSummerTotalExc.year), locationFun = 1, threshold = threshold, getParams = True, returnPeriod = 20, returnValue = 10, xNew = yrsToPred, bootSE = True, bootControl = {'n':150, 'seed':3}, getFit = True)
# result['info']   # information on the optimization
# result['info']['counts']  # number of evaluations in the optimization
# result['info']['counts_names'] # names to interpret 'counts'
# result['numBootFailures']      # number of bootstrap datasets for which the model could not be fit; if this is non-negligible relative to the number of bootstrap samples (default of 250), interpret the bootstrap results with caution

# # result['fit']  # voluminous output from the R function that does the fitting

return value is:  [38.00563858]
[2.93335284]
